First connect through SSH tunnel to tools-login.wmflabs.org server.

Replace the username and password (below) with those you find in the replica.my.cnf file in the host home

In [1]:
import pandas as pd
import numpy as np
import pymysql                                               # to get, do pip install PyMySQL
import json
from sklearn.feature_extraction.text import CountVectorizer  #from the NLP tutorial
import re                                                    #from NLP tutorial

In [2]:
#This connects to the Wiki database

username = 'u15068'                # get from replica.my.cnf
password = 'G5V2xrWbdFTSkyoU'      # get from replica.my.cnf
db=pymysql.connect(host='localhost',port=4711,user=username,passwd=password,db='enwiki_p')

In [3]:
#Get the epa id numbers from the dataset on github

epa_articles = pd.read_csv("070916_edit_protected_articles.csv")
print(epa_articles.shape, len(epa_articles))
epa_ids=epa_articles["page_id"]
epa_ids[300:305]
epa_ids[7]

(7526, 20) 7526


628

In [14]:
#Quary the wiki db for caterogylinks a page_id at a time
#creates the dictionary data, in the format {"page_id" : ["link1", "link2", ...]} for all edit-protected articles

data={}
for i in range(len(epa_ids)):       #to get them all len(epa_articles)
    cur = db.cursor()                  #this is how you use pymusql apparently
    page_id=epa_ids[i]
    if i %1000 ==0:
        print("Quarry article", epa_ids[i])
    quarry="""
    SELECT page_id, cl_to
    FROM page inner join categorylinks on(page.page_id = categorylinks.cl_from)
    WHERE page_id = %s""" 
    cur.execute(quarry, np.asscalar(epa_ids[i]))
    link = np.array(cur.fetchall())
    if link.size is 0:
        print("Empty categorylink %s", epa_ids[i])
    else:   
        key=link[0,0].decode("utf-8")
        data[key]=[link[l,1].decode("utf-8") for l in range(len(link[:,1]))]
    
    #link = np.array(cur.fetchall())
    #data.append(link.tolist())
    #data=np.concatenate((data,link), axis=0)
    #Takes 8minuts to 10 min
    #link[:,1].tolist() this is to send an array in a list

Quarry article 25
Quarry article 39988
Quarry article 326123
Quarry article 1331018
Empty categorylink %s 4485200
Quarry article 4553108
Quarry article 13253481
Quarry article 24572232
Quarry article 41698932
Empty categorylink %s 15580374


In [15]:
len(data)

7524

In [16]:
link

array([[b'50730392', b'1990_births'],
       [b'50730392', b'All_articles_needing_additional_references'],
       [b'50730392', b'Articles_for_deletion'],
       [b'50730392',
        b'Articles_needing_additional_references_from_June_2016'],
       [b'50730392', b'Articles_to_be_expanded_from_June_2016'],
       [b'50730392', b'Astralis_players'],
       [b'50730392', b'Counter-Strike_players'],
       [b'50730392', b'Danish_eSports_players'],
       [b'50730392', b'Fnatic_players'],
       [b'50730392', b'Living_people'],
       [b'50730392', b'Mousesports_players'],
       [b'50730392', b'Team_Dignitas_players'],
       [b'50730392', b'Team_SoloMid_players']], 
      dtype='|S53')

In [17]:
with open('cl_epa.txt', 'w') as outfile:
    json.dump(data, outfile)

The Bag of words tool needs a list of words to be trained on, so I extract that from the dictionary data

In [ ]:
#I need to check that this works on the datashape I produce
clean_train_categories = []
for key in data:                                                               
    clean_train_categories.append(data[key]) #this way I get only the list of categories without the keys

In [ ]:
len(data_dic)

In [ ]:
# Look into the parameters of the vectorizer, stop_words I could use!
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_categories)
print(type(train_data_features))
print(train_data_features.shape)
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
print(type(train_data_features))
train_data_features.shape

In [ ]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab[1000:1100])

In [ ]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for i,j in zip(vocab, dist):
    if j>500:
        print(j,i)

In [ ]:
import matplotlib.pyplot as plt
dist.shape

In [ ]:
hist, bins = np.histogram(dist,bins=np.logspace(1.4, 3.2, 50), range=(0,4000), density=False)
hist[7]    #around bin#7 I have hit 500 less common categories, that corresponds to about an occurrency of 140

In [ ]:
#shouldn't be an easier way to plot an histogram?
%matplotlib inline 
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.gca().set_xscale("log")
plt.ylabel('CategoryLinks')
plt.xlabel('Occurrencies')
plt.show()

In [ ]:
#remove useless words?? I can I judge that? The algorithm will tell what is really useless, right?!
###This is the point of Machine learning... I don't have to do the work, but the MACHINE!

#then I would do the same thing, but parsing the words from the categories separately
#Then the same thing but including all the links in the article from the other table
#and other links: formatting?

#one json for categorylinks, with one key per article and a table of elements (links)
#I should organize everything in functions and push the code to github
#

In [ ]:

letters_only = re.sub("[^a-zA-Z]", " ", data_dic['307'][2] ) #this is a way to get only letters and remove simbols and underscore
print(letters_only)
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words
print(words[:20])
merge=" ".join(words)
merge

In [ ]:
######___PYTHON__NOTES___######

#this is how you concatenate np arrays

try1=np.array([])
a1=np.array([1,2,3])
a2=np.array([4,5,6])
try1=np.concatenate((a1,a2),axis=0)
try1

#to create a dictionary

data_dic={}
current_id="0"                                     #CAVEAT: make sure the first page_id is not "0"
for item in range(len(data)):                             #Iterate through the all the categorylink table
    page_id=data[item][0].decode("utf-8")          #estract the page_id from the first column
    if page_id == current_id :                     #page_id not new
        table.append(data[item][2].decode("utf-8"))
        data_dic[page_id]=table
    else:                                          #new page_id
        table=[]
        table.append(data[item][2].decode("utf-8"))
        data_dic[page_id]=table                    #add value to the table and the dictionary
        current_id=page_id


#json_data=json.dumps(data_dic) #this dumps the dictionary into a json-string that can be .loads back into a json-object
#retrieve with json.loads()
#print(json_data)

#if I want to take elements of a list and merge them in a string
for key in data_dic:
    data_dic[key]=" ".join(data_dic[key])
data_dic

You can keep the really specific topics that occur above some threshold of pages (5%? 10%?), then split the topics into words, filter stop words, and the bag of words to get things that are common enough to model.


In [ ]:
#I should try and run my own random forest, but first
#I should get a sample of non edit-protected articles (what if they have other kinds of protection?)
#then mix them with my dataset, make sure they are labelled, split in training and validation sets 
#train a random forest algorithm
#To do that I will have to modify the query and I might want to branch off this notebook, so first I should push it on github 
#And this is a good time to re-practice that